In [1]:
import gensim
import os
import collections
import smart_open
import random

C:\Users\t-wajian\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Slow version of gensim.models.doc2vec is being used


In [43]:
train_path = '../test_data_1462/'

In [44]:
def read_corpus(path, tokens_only=False):
    for name in os.listdir(train_path):
        fname = os.path.join(train_path, name)
        with smart_open.smart_open(fname) as f:
            for i, line in enumerate(f):
                if tokens_only:
                    yield gensim.utils.simple_preprocess(line)
                else:
                    # For training data, add tags
                    yield gensim.models.doc2vec.TaggedDocument([gensim.utils.any2unicode(i) for i in line.split()[1:]], line.split()[0])

In [45]:
train_corpus = list(read_corpus(train_path))

In [50]:
train_corpus[0]

TaggedDocument(words=['market.hiapk.com', 'configsvr.msf.3g.qq.com', 'market.hiapk.com', '120.198.199.150', 'plus.xiangji.qq.com', 'mreg.kuwo.cn', 'conf.3g.qq.com', 'plus.xiangji.qq.com', 'conf.3g.qq.com', 'www.umeng.com', 'qlogo1.store.qq.com', 'q1.qlogo.cn', 'mreg.kuwo.cn', 'www.umeng.com', 'item.kakao.com', 'st-talk.kakao.com', 'fr-talk.kakao.com', 'plus-talk.kakao.com', 'www.umeng.com', 'www.umeng.co', 'www.umeng.com', 'monitor.uu.qq.com', 'plus.xiangji.qq.com', 'conf.3g.qq.com', 'plus.xiangji.qq.com', 'conf.3g.qq.com', 'plus.xiangji.qq.com', 'fr-talk.kakao.com', 'st-talk.kakao.com', 'monitor.uu.qq.com', 'st-talk.kakao.com', 'plus.xiangji.qq.com', 'conf.3g.qq.com', 'plus.xiangji.qq.com', 'conf.3g.qq.com', 'www.umeng.com', 'item.kakao.com', 'st-talk.kakao.com', 'plus-talk.kakao.com', 'plus.xiangji.qq.com', 'st-talk.kakao.com', 'plus-talk.kakao.com', 'fr-talk.kakao.com', 'plus-talk.kakao.com', 'pf.talk.kakao.co.kr', 'plus.xiangji.qq.com', 'conf.3g.qq.com', 'plus.xiangji.qq.com', 'con

In [51]:
import multiprocessing
cores = multiprocessing.cpu_count()

model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55, workers=cores)

In [52]:
model.build_vocab(train_corpus)

In [54]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

C:\Users\t-wajian\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\models\word2vec.py:787: UserWarning: C extension not loaded for Word2Vec, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn("C extension not loaded for Word2Vec, training will be slow. "


Wall time: 14h 16min 59s


8842716

In [58]:
model.infer_vector(['www.baidu.com'])

array([ 0.11527336, -0.08250662,  0.1576063 , -0.05272446, -0.01677831,
        0.07801579,  0.02565984,  0.02580809, -0.10853845,  0.15178277,
       -0.04096612, -0.00697888, -0.01375838, -0.15601423,  0.10014983,
        0.07820816, -0.0693801 , -0.1444713 ,  0.03140168, -0.15862148,
        0.07860304,  0.12365799,  0.00408718, -0.0142957 ,  0.13812371,
       -0.01755551, -0.00222325,  0.10454232,  0.07087982,  0.03346878,
       -0.03766483, -0.14109099, -0.03278403,  0.09308196, -0.05890558,
        0.05948442,  0.07872929, -0.04301341, -0.03888891,  0.00493651,
       -0.18817024,  0.04812083, -0.1263907 , -0.04586846,  0.03200846,
        0.04501254,  0.14455464, -0.04802207, -0.0172881 ,  0.10397528], dtype=float32)

In [69]:
model.infer_vector(['only'])

array([ -9.71526955e-04,  -6.25042652e-04,   6.48722798e-03,
        -5.97501220e-03,   6.19377941e-04,   1.16862159e-03,
         7.70496484e-03,   6.79999869e-03,  -1.19464588e-03,
        -8.53012409e-03,   3.67454626e-03,   9.42977238e-03,
         6.54420815e-03,  -8.19337438e-04,  -9.62834060e-03,
         7.76772154e-03,   2.18170951e-03,   9.24124010e-03,
         9.84979142e-03,   9.02022410e-04,  -2.31309430e-04,
        -9.09298033e-05,   5.03535150e-03,  -3.02261906e-03,
        -1.98686984e-03,   4.75452421e-03,   8.27871356e-03,
         7.64060114e-03,   9.09098331e-03,   8.56771693e-03,
         3.24668270e-03,  -6.17854716e-03,   4.84256493e-03,
         7.34344078e-03,  -4.45673149e-03,  -6.52507879e-03,
         6.55823248e-03,   1.68913894e-03,   9.89754219e-03,
         6.44636247e-03,   2.80873314e-03,  -9.70168319e-03,
        -8.25118646e-03,  -7.15151802e-03,  -9.12653282e-03,
         3.61963036e-03,   1.37391000e-03,   2.30686064e-03,
         2.46210839e-03,

In [78]:
ranks = []
second_ranks = []
for doc_id in range(100):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=2)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    second_ranks.append(sims[1])

In [79]:
collections.Counter(ranks)

Counter({0: 99, 1: 1})

In [80]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (99): «init-p01st.push.apple.com monitor.uu.qq.com conf.3g.qq.com monitor.uu.qq.com configsvr.msf.3g.qq.com jsync.3g.qq.com pub.idqqimg.com pnewsapp.tc.qq.com monitor.uu.qq.com m.qpic.cn ugc.qpic.cn m.qpic.cn ugc.qpic.cn m.qpic.cn short.weixin.qq.com mmsns.qpic.cn init-p01st.push.apple.com fishingjoy.punchbox.org uapi.punchbox.org fishingjoy.punchbox.org fishingjoy2.appget.cn data.flurry.com fishingjoy.punchbox.org fishingjoy2.appget.cn sandbox.gc.apple.com fishingjoy2.appget.cn data.flurry.com down.qq.com 120.209.135.192 cgi.connect.qq.com cgi.qplus.com monitor.uu.qq.com configsvr.msf.3g.qq.com conf.3g.qq.com monitor.uu.qq.com pub.idqqimg.com kiss.3g.qq.com pub.idqqimg.com kiss.3g.qq.com qzs.qq.com rd.go.10086.cn ax.init.itunes.apple.com 120.209.135.192 monitor.uu.qq.com opensdk.uu.qq.com q1.qlogo.cn appimg1.3g.qq.com monitor.uu.qq.com web.g.qq.com iyouxi.vip.qq.com gamecenter.qq.com web.g.qq.com imgcache.qq.com isdspeed.qq.com web.g.qq.com q4.qlogo.cn m.qpic.cn init-p01st.pu

In [96]:
gensim.models.ldamulticore.LdaMulticore(corpus=[i[0] for i in train_corpus[:100]], alpha= 0.001, eta = 0.001,id2word=None, num_topics=10, offset = 50.0, chunksize=10000)

ValueError: too many values to unpack (expected 2)

In [9]:
gensim.models.ldamulticore.LdaMulticore?

In [10]:
[i[0] for i in train_corpus[:2]]

[['mmsc.monternet.com',
  'conf.3g.qq.com',
  'qlogo1.store.qq.com',
  'm.qpic.cn',
  'wspeed.qq.com',
  'activeqq.3g.qq.com',
  'pub.idqqimg.com',
  '3gimg.qq.com',
  'activeqq.3g.qq.com',
  '3gimg.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'conf.3g.qq.com',
  'm.qpic.cn',
  'p.store.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'conf.3g.qq.com',
  'm.qpic.cn',
  'p.store.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'monitor.uu.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  'm.qpic.cn',
  'plus.xiangji.qq.com',
  '10.0.0.172',
  'wspeed.qq.com',
  'monitor.uu.qq.com',
  'wspeed.qq.com',
  'p.store.qq.com',
  '117.135.171.215',
  'wspeed.qq.com',
  'info60.z.qq.com',
  '120.203.230.182',
  'v.gdt.qq.com',
  'a258.photo.store.qq.com',
  'a127.photo.store.qq.com',
  'a258.photo.store.qq.com',
  '120.203.230.182',
  'blog60.z.qq.com',
  'v.gdt